### si tratta di riprodurre alcuni risultati già trovati con Patstat online tramite SQLAlchemy. Alcune strategie sono comunque utili per analisi di Patent Intelligence.

In [1]:
# Importing the patstat client
from epo.tipdata.patstat import PatstatClient

# Initialize the PATSTAT client
patstat = PatstatClient(env='PROD')

# Access ORM
db = patstat.orm()

# Importing tables as models
from epo.tipdata.patstat.database.models import TLS201_APPLN,TLS203_APPLN_ABSTR, TLS202_APPLN_TITLE, TLS209_APPLN_IPC, TLS224_APPLN_CPC

# Ora trattiamo un caso reale

Step 1: Retrieve distinct docdb_family_id values corresponding to keywords

In [2]:
from sqlalchemy import and_, or_
# Keywords for the abstract and title
keywords = [
    "rare earth element*", "light REE*", "heavy REE*", "rare earth metal*",
    "rare earth oxide*", "lanthan*", "rare earth"
]
# Step 1: Retrieve distinct docdb_family_id values corresponding to keywords
subquery_keywords = (
    db.query(TLS201_APPLN.docdb_family_id)
    .join(TLS203_APPLN_ABSTR, TLS203_APPLN_ABSTR.appln_id == TLS201_APPLN.appln_id)
    .filter(or_(*[TLS203_APPLN_ABSTR.appln_abstract.contains(kw) for kw in keywords]))
    .union(
        db.query(TLS201_APPLN.docdb_family_id)
        .join(TLS202_APPLN_TITLE, TLS202_APPLN_TITLE.appln_id == TLS201_APPLN.appln_id)
        .filter(or_(*[TLS202_APPLN_TITLE.appln_title.contains(kw) for kw in keywords]))
    ).distinct()
).all()

# Convert result to a list of docdb_family_id values
docdb_family_ids_keywords = [row.docdb_family_id for row in subquery_keywords]
# Print the number of distinct docdb_family_id values
print(f"Number of distinct docdb_family_id values: {len(docdb_family_ids_keywords)}")

Number of distinct docdb_family_id values: 84905


Step 2: Retrieve distinct docdb_family_id values corresponding to classification codes

In [3]:
from sqlalchemy import func

# Classification codes with varying lengths

ipc_codes_11 = [
'A43B   1/12','B03B   9/06','B29B   7/66','B30B   9/32','B65D  65/46','C03B   1/02',
'C04B   7/24','C04B   7/26','C04B   7/28','C04B   7/30','C04B  11/26','C04B  18/04','C04B  18/06','C04B  18/08','C04B  18/10',
'C04B  18/12','C04B  18/14','C04B  18/16','C04B  18/18','C04B  18/20','C04B  18/22','C04B  18/24','C04B  18/26','C04B  18/28',
'C04B  18/30','C09K  11/01','C22B  19/28','C22B  19/30','C22B  25/06','D21B   1/08','D21B   1/10','D21B   1/32','D21C   5/02',
'D21H  17/01','H01B  15/00','H01J   9/52','H01M   6/52','H01M  10/54']

ipc_codes_8 = [
'B22F   8','B29B  17','B62D  67','B65H  73',
'C08J  11','C10M 175','C22B   7','D01G  11']

ipc_codes_12 = [
'C04B  33/132']

cpc_codes_11 = ['A43B   1/12','B03B   9/06','B29B   7/66','B30B   9/32','B65D  65/46','C03B   1/02',
'C04B   7/24','C04B   7/26','C04B   7/28','C04B   7/30','C04B  11/26','C04B  18/04','C04B  18/06','C04B  18/08','C04B  18/10',
'C04B  18/12','C04B  18/14','C04B  18/16','C04B  18/18','C04B  18/20','C04B  18/22','C04B  18/24','C04B  18/26','C04B  18/28',
'C04B  18/30','C09K  11/01','C22B  19/28','C22B  19/30','C22B  25/06','D21B   1/08','D21B   1/10','D21B   1/32','D21C   5/02',
'D21H  17/01','H01B  15/00','H01J   9/52','H01M   6/52','H01M  10/54','Y02W  30/50','Y02W  30/52','Y02W  30/56','Y02W  30/58',
'Y02W  30/60','Y02W  30/62','Y02W  30/64','Y02W  30/66','Y02W  30/74','Y02W  30/78','Y02W  30/80','Y02W  30/82','Y02W  30/84',
'Y02W  30/91','Y02P  10/20']

cpc_codes_8 = ['B22F   8','B29B  17','B62D  67','B65H  73',
'C08J  11','C10M 175','C22B   7','D01G  11']

cpc_codes_12 = ['C04B  18/068','C04B  33/132',
'C04B   7/243','C04B   7/246','C04B  18/049','C04B  18/061','C04B  18/062','C04B  18/064','C04B  18/065',
'C04B  18/067','C04B  18/081','C04B  18/082','C04B  18/084','C04B  18/085','C04B  18/087','C04B  18/088',
'C04B  18/101','C04B  18/103','C04B  18/105','C04B  18/106','C04B  18/108','C04B  18/125',
'C04B  18/141','C04B  18/142','C04B  18/143','C04B  18/144','C04B  18/145','C04B  18/146','C04B  18/147',
'C04B  18/148','C04B  18/149','C04B  18/162','C04B  18/165','C04B  18/167','C04B  18/241','C04B  18/243',
'C04B  18/245','C04B  18/246','C04B  18/248','C04B  18/265','C04B  18/305']


cpc_codes_13 = ['C04B  18/0409','C04B  18/0418',
'C04B  18/0427','C04B  18/0436','C04B  18/0445','C04B  18/0454','C04B  18/0463','C04B  18/0472','C04B  18/0481']





# Subquery for classification codes with varying lengths
subquery_classcodes = (
    db.query(TLS201_APPLN.docdb_family_id)
    .join(TLS209_APPLN_IPC, TLS209_APPLN_IPC.appln_id == TLS201_APPLN.appln_id)
    .filter(
        or_(
            func.substr(TLS209_APPLN_IPC.ipc_class_symbol, 1, 11).in_(ipc_codes_11),
            func.substr(TLS209_APPLN_IPC.ipc_class_symbol, 1, 8).in_(ipc_codes_8),
            func.substr(TLS209_APPLN_IPC.ipc_class_symbol, 1, 12).in_(ipc_codes_12)
        )
    )
    .union(
        db.query(TLS201_APPLN.docdb_family_id)
        .join(TLS224_APPLN_CPC, TLS224_APPLN_CPC.appln_id == TLS201_APPLN.appln_id)
        .filter(
            or_(
                func.substr(TLS224_APPLN_CPC.cpc_class_symbol, 1, 11).in_(cpc_codes_11),
                func.substr(TLS224_APPLN_CPC.cpc_class_symbol, 1, 8).in_(cpc_codes_8),
                func.substr(TLS224_APPLN_CPC.cpc_class_symbol, 1, 12).in_(cpc_codes_12),
                func.substr(TLS224_APPLN_CPC.cpc_class_symbol, 1, 12).in_(cpc_codes_13)
            )
        )
    ).distinct()
).all()

# Convert result to a list of docdb_family_id values
docdb_family_ids_classcodes = [row.docdb_family_id for row in subquery_classcodes]

# Print the number of results
print(f"Number of results in Step 2: {len(docdb_family_ids_classcodes)}")

Number of results in Step 2: 567012


Step 3: Get the intersection of the two lists

In [4]:
# Step 3: Get the intersection of the two lists
intersection_docdb_family_ids = list(set(docdb_family_ids_keywords) & set(docdb_family_ids_classcodes))

Step 4: Retrieve and display the list of results

In [5]:
from sqlalchemy.orm import aliased
from sqlalchemy import and_, func
import pandas as pd

# Assuming you have the necessary imports for your database models
from epo.tipdata.patstat.database.models import TLS209_APPLN_IPC, TLS201_APPLN

# Alias TLS209_APPLN_IPC to avoid confusion
TLS209_APPLN_IPC_2 = aliased(TLS209_APPLN_IPC)

# Step 4: Filter the data according to the timeframe 2010 - 2022 and join with TLS209_APPLN_IPC
final_query = (
    db.query(
        TLS201_APPLN.docdb_family_id.label('patent_family_id'),
        TLS201_APPLN.earliest_filing_year.label('earliest_filing_year'),
        TLS209_APPLN_IPC.ipc_class_symbol.label('IPC_1'),
        TLS209_APPLN_IPC_2.ipc_class_symbol.label('IPC_2')
    )
    .join(TLS209_APPLN_IPC, TLS201_APPLN.appln_id == TLS209_APPLN_IPC.appln_id)
    .join(TLS209_APPLN_IPC_2, TLS201_APPLN.appln_id == TLS209_APPLN_IPC_2.appln_id)
    .filter(TLS201_APPLN.docdb_family_id.in_(intersection_docdb_family_ids))
    .filter(TLS201_APPLN.earliest_filing_year.between(2010, 2022))
    .filter(and_(
        TLS209_APPLN_IPC.ipc_class_symbol > TLS209_APPLN_IPC_2.ipc_class_symbol,
        func.left(TLS209_APPLN_IPC.ipc_class_symbol, 8) > func.left(TLS209_APPLN_IPC_2.ipc_class_symbol, 8)
    ))
)

# Execute the query
result = final_query.all()

# Convert the result to a DataFrame
df = pd.DataFrame(result, columns=['patent_family_id', 'earliest_filing_year', 'IPC_1', 'IPC_2'])

# Ensure that IPC classification codes are strings before truncating
df['IPC_1'] = df['IPC_1'].astype(str).apply(lambda x: x[:8])
df['IPC_2'] = df['IPC_2'].astype(str).apply(lambda x: x[:8])

# Display the DataFrame
print(df)

       patent_family_id  earliest_filing_year     IPC_1     IPC_2
0              55530492                  2015  C22B   7  C22B   3
1              55530492                  2015  C22B  59  C22B   3
2              67817265                  2019  G06T   7  G06F  30
3              67817265                  2019  G06N   3  G06F  30
4              67817265                  2019  G06N   3  G06F  30
...                 ...                   ...       ...       ...
51310          84858887                  2022  C22C  38  C22C  33
51311          84858887                  2022  C22C  38  C22C  33
51312          84858887                  2022  C22C  38  C22C  33
51313          84858887                  2022  C22C  38  C22C  33
51314          84858887                  2022  C22C  38  C22C  33

[51315 rows x 4 columns]


In [6]:
import pandas as pd

# Assuming df is the DataFrame we have just created
# Calculate the number of duplicated rows
num_duplicates = df.duplicated().sum()

# Remove duplicate rows
df_unique = df.drop_duplicates()

# Print the number of duplicated rows deleted
print(f"Number of duplicated rows deleted: {num_duplicates}")

# Group by IPC_1, IPC_2, and earliest_filing_year and count the number of families
df_grouped = df_unique.groupby(['IPC_1', 'IPC_2', 'earliest_filing_year']).size().reset_index(name='count_of_families')

# Display the new DataFrame
print(df_grouped)
# Save the DataFrame to an Excel file
df_grouped.to_excel("grouped_data.xlsx", index=False)

print("The grouped data has been saved to 'grouped_data.xlsx'.")

Number of duplicated rows deleted: 29778
          IPC_1     IPC_2  earliest_filing_year  count_of_families
0      A01G  17  A01B  79                  2021                  1
1      A01G  20  A01G   9                  2016                  1
2      A01G  22  A01B  79                  2021                  1
3      A01G  22  A01G  17                  2021                  1
4      A01H   6  A01H   5                  2022                  1
...         ...       ...                   ...                ...
10589  H05B   6  B03C   1                  2015                  1
10590  H05B   6  B03C   7                  2015                  1
10591  H05B   6  B23K  26                  2015                  1
10592  H05B   6  H01F  13                  2015                  1
10593  H10N  35  H01M  10                  2017                  3

[10594 rows x 4 columns]
The grouped data has been saved to 'grouped_data.xlsx'.


In [7]:
import pandas as pd
import plotly.express as px
import plotly.offline as pyo

# Assuming df is the DataFrame we have just created
# Calculate the number of duplicated rows
num_duplicates = df.duplicated().sum()

# Remove duplicate rows
df_unique = df.drop_duplicates()

# Print the number of duplicated rows deleted
print(f"Number of duplicated rows deleted: {num_duplicates}")

# Normalize IPC codes to ensure consistent ordering
df_unique['IPC_pair'] = df_unique.apply(lambda row: tuple(sorted([row['IPC_1'], row['IPC_2']])), axis=1)

# Group by IPC_pair and earliest_filing_year and count the number of families
df_grouped = df_unique.groupby(['IPC_pair', 'earliest_filing_year']).size().reset_index(name='count_of_families')

# Split IPC_pair back into IPC_1 and IPC_2 for plotting
df_grouped[['IPC_1', 'IPC_2']] = pd.DataFrame(df_grouped['IPC_pair'].tolist(), index=df_grouped.index)

# Function to create and save scatter plot for a given year range with a threshold for count_of_families
def create_scatter_plot(df, start_year, end_year, filename, threshold=5):
    # Filter the data for the given year range
    df_filtered = df[(df['earliest_filing_year'] >= start_year) & (df['earliest_filing_year'] <= end_year)]
    
    # Aggregate the counts across the year range
    df_aggregated = df_filtered.groupby(['IPC_1', 'IPC_2']).agg({'count_of_families': 'sum'}).reset_index()
    
    # Apply the threshold
    df_aggregated = df_aggregated[df_aggregated['count_of_families'] >= threshold]
    
    # Create the scatter plot
    fig = px.scatter(df_aggregated, 
                     x='IPC_1', 
                     y='IPC_2', 
                     size='count_of_families', 
                     color='count_of_families',
                     labels={'IPC_1': 'IPC1 Codes', 'IPC_2': 'IPC2 Codes', 'count_of_families': 'Count of Families'},
                     title=f'Scatter Plot of IPC Codes and Family Counts for {start_year}-{end_year} (Threshold: {threshold} Families)')
    
    # Update layout to resize the vertical axis to let all classification codes' legends appear
    fig.update_layout(
        yaxis=dict(
            automargin=True,
            tickmode='array',
            tickvals=df_aggregated['IPC_2'].unique(),  # Use unique IPC_2 values as tick values
            ticktext=df_aggregated['IPC_2'].unique()  # Ensure all IPC_2 values are shown
        )
    )
    
    # Save the plot as an HTML file
    pyo.plot(fig, filename=filename)
    print(f"The scatter plot has been saved as '{filename}'.")

# Create scatter plots for the specified year ranges with a threshold of 5 families
create_scatter_plot(df_grouped, 2012, 2017, 'scatter_plot_2012_2017.html', threshold=5)
create_scatter_plot(df_grouped, 2018, 2023, 'scatter_plot_2018_2023.html', threshold=5)

Number of duplicated rows deleted: 29778


/tmp/ipykernel_57145/3844695591.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



The scatter plot has been saved as 'scatter_plot_2012_2017.html'.
The scatter plot has been saved as 'scatter_plot_2018_2023.html'.
